In [8]:
import pandas as pd

In [9]:
def Normalize(temp_dataset, nCol, weights):
    # normalizing the array
    # normalizes all the data between 0 and 1
    # print(" Normalizing the DataSet...\n")
    for i in range(1, nCol):
        temp = 0
        for j in range(len(temp_dataset)):
            temp = temp + temp_dataset.iloc[j, i]**2
        temp = temp**0.5
        for j in range(len(temp_dataset)):
            temp_dataset.iat[j, i] = (
                temp_dataset.iloc[j, i] / temp)*weights[i-1]
    return temp_dataset

In [10]:

def Calc_Values(temp_dataset, nCol, impact):
    # print(" Calculating Positive and Negative values...\n")
    p_sln = (temp_dataset.max().values)[1:]
    n_sln = (temp_dataset.min().values)[1:]
    for i in range(1, nCol):
        if impact[i-1] == '-':
            p_sln[i-1], n_sln[i-1] = n_sln[i-1], p_sln[i-1]
    return p_sln, n_sln


In [54]:
def topsis(temp_dataset, dataset, nCol, weights, impact):
    # brings everything together
    # normalizing the array
    temp_dataset = Normalize(temp_dataset, nCol, weights)

    # Calculating positive and negative values
    p_sln, n_sln = Calc_Values(temp_dataset, nCol, impact)

    # calculating topsis score
    # print(" Generating Score and Rank...\n")
    score = []
    for i in range(len(temp_dataset)):
        temp_p, temp_n = 0, 0
        for j in range(1, nCol):
            temp_p = temp_p + (p_sln[j-1] - temp_dataset.iloc[i, j])**2
            temp_n = temp_n + (n_sln[j-1] - temp_dataset.iloc[i, j])**2
        temp_p, temp_n = temp_p**0.5, temp_n**0.5
        score.append(temp_n/(temp_p + temp_n))
    dataset['TOPSIS Score'] = score

    # calculating the rank according to topsis score
    dataset['Rank'] = (dataset['TOPSIS Score'].rank(
        method='max', ascending=True))
    dataset = dataset.astype({"Rank": int})

    print("Results:")
    print("CI", dataset)

    # dataset.to_csv("results.csv", index=False)
    dataset.to_csv("results.csv", index=True, index_label="Country")


In [68]:
data = pd.read_csv("data.csv")
data.set_index('Country', inplace=True)
newData = pd.DataFrame()


print("Dummy Data:")
print(data)
print("")


nCol = len(data.columns.values)
weights = [10, 1, 1]
impact = ['+', '+', '-']

# need to show country in result
# test with large dataset
results = topsis(data, newData, nCol, weights, impact)




Dummy Data:
                 HDI  Education  Inequality
Country                                    
Afghanistan    0.511      0.414        28.2
Australia      0.944      0.924         8.0
Brazil         0.765      0.694        23.8
Germany        0.947      0.943         7.9
United States  0.926      0.900        12.4

Results:
CI    TOPSIS Score  Rank
0      0.144988     1
1      0.848215     3
2      0.536144     2
3      0.855012     4
4      0.855739     5
